### Example tree (pre-order traversing thread)
![Tree in this example](tree.png)

In [1]:
# Node class
class Node(object):
    def __init__(self, s):
        self.val = s
        self.left = None
        self.right = None

# Serilized trees, "#" marks a None child
pre_order = ["F","B","A","#","#","D","C","#","#","E","#","#","G","#","I","H","#","#","#"]
in_order = ["#","A","#","B","#","C","#","D","#","E","#","F","#","G","#","H","#","I","#"]
post_order = ["#","#","A","#","#","C","#","#","E","D","B","#","#","#","H","#","I","G","F"]
level_order = ["F","B","G","A","D","#","I","#","#","C","E","H","#","#","#","#","#","#","#"]

In [2]:
# Practice: pre-order serializer and deserializer

# Serializer
def se_preorder(root):
    
    # Inner recursive function 
    def recur_se(root, lst):
        if root == None:
            lst.append("#")
            return
        else:
            lst.append(root.val)
            recur_se(root.left, serialized)
            recur_se(root.right, serialized)
    
    serialized = []
    recur_se(root, serialized)
    return serialized

# De-serializer
def de_preorder(serialized_val):
    
    # Inner deserializer recursivelt built small triangle tree-like structure.
    #
    #                     root <- index=i
    #                    /   \
    #  index = i+1 -> left  right <- index = 1+index of (the rightmost node in left children's tree)
    #
    # Assume root node's at serialized[i], now we start to build the whole tree
    # First step in recursion is to build the root, which is easy because we have its index i 
    # Next step is also easy because we know that the left's index is root's + 1
    # The right is harder but by we know that recursiive building of right should happen after resusive building left tree
    # Therefore the recurison of left tree should return the right's index which is 1+(index of last node in left tree)
    # Now we have rebuilt the tree, what's left is just to return the tree
    # 
    # BUT! Don't forget it's recursion. Since to build the right tree need its own index returned
    # ,therefore the whole recursion need to return the index of the next node that recursion should happen on
    # Are we done? No! Again, we need to deal with None node. It's easy thought since None node doesn't has children
    # So None node doesn't have children to rebuild. Therefore we just return None and the next index. That's it!
    def recur_de(lst, i):
        root_val = lst[i]
        
        # When the val shows it's a None node, just return
        if root_val == "#":
            return (None, i+1)
        
        # When this value is a real node
        root = Node(root_val)
        # Recurse left and right to find their trees
        left, right_i = recur_de(lst, i+1)
        right, next_i = recur_de(lst, right_i)
        # Link left and right trees to root
        root.left = left
        root.right = right
        
        return root, next_i
    
    root, length = recur_de(serialized_val, 0)
    
    return root

In [3]:
# Test pre-order serializer and de-serializer
print pre_order == se_preorder(de_preorder(pre_order))

True


In [4]:
# Priactice: post-order serializer and de-serializer
def se_postorder(root):
    
    def recur_se(root, lst):
        if root == None:
            lst.append("#")
        else:
            recur_se(root.left, lst)
            recur_se(root.right, lst)
            lst.append(root.val)
    
    lst =[]
    recur_se(root, lst)
    return lst

def de_postorder(lst):
    
    # de-serialize from back
    def recur_de(lst, pos):
        if lst[pos] == "#":
            return None, pos-1
        
        else:
            root = Node(lst[pos])
            right, left_pos = recur_de(lst, pos-1)
            left, next_pos = recur_de(lst, left_pos)
            root.left = left
            root.right = right
            
            return root, next_pos
    
    root, length = recur_de(lst, len(lst)-1)
    return root

In [7]:
# Test:post-order serializer and de-serializer
print post_order == se_postorder(de_postorder(post_order))

True


In [46]:
# Practice: level-order serializer and de-serializer

# Typical BFS, choose the order from left to right
from collections import deque
def se_bfs(root):
    q = deque()
    q.append(root)
    serialized = []
    
    while len(q) > 0:
        root = q.popleft()
        
        if root == None:
            return []
        elif root == "#":
            serialized.append("#")
            continue
        else:
            serialized.append(root.val)
        
        if root.left != None:
            q.append(root.left)
        else:
            q.append("#")
        if root.right != None:
            q.append(root.right)
        else:
            q.append("#")
    
    return serialized

# de-serialize

def de_lvlorder(serialized):
    # An amazing fact is the if level-order serialization fills in each None child

In [47]:
# Test leve-order serialize and de-serialize
print level_order == se_lvlorder(de_lvlorder(level_order))

['F', 'B', 'G', 'A', 'D', '#', 'I', '#', '#', 'C', 'E', 'H', '#', '#', '#', '#', '#', '#', '#']



In [9]:
# Practice: iterative traverse a tree pre-order

def iter_preorder(root):
    track = []
    s = list()
    s.append(root)
    while len(s) > 0:
        # 因为搜索tree和访问node都在root上完成，
        # 所以一步接触root就完成它地全部使命，所以可以直接pop
        root = s.pop()
        track.append(root.val) # visit the root
        
        # 如果right存在，加入stack等待被访问
        if root.right != None:
            s.append(root.right)
        
        # 如果left存在，加入stack等待被访问
        # left是后加地所以会被先pop掉
        if root.left != None:
            s.append(root.left)
    
    return track

In [10]:
# Test iterative travers a tree pre-order
root = de_preorder(pre_order)
track = iter_preorder(root)
print track

['F', 'B', 'A', 'D', 'C', 'E', 'G', 'I', 'H']


In [29]:
# Practice: iterative traverse a tree post-order

def iter_postorder(root):
    track = []
    s = list()
    s.append(root)
    visited = set([None]) # ensure that when reached the most left, the while loop starts vi
    
    while len(s) > 0:
        # 有两个任务需要完成，第一个是搜索tree，第二个是访问Node
        root = s[-1]
        
        if root == None:
            return []
        
        # 首先搜索，因为后来要先访问左边，再访问右边
        # 所以就先搜索右边，在搜索左边
        if root.right not in visited:
            s.append(root.right)
        if root.left not in visited:
            s.append(root.left)
        
        # 然后是访问,只有当它的left和right都被访问了,
        # 或者left/right是None所以没有访问一说，才可以访问它
        if (root.left in visited) and (root.right in visited):
            s.pop() # remove from stack
            track.append(root.val) # 访问
            visited.add(root) # 添加到已访问
    
    return track

In [28]:
# Test iterative travers a tree post-order
root = de_preorder(pre_order)
track = iter_postorder(root)
print track

['A', 'C', 'E', 'D', 'B', 'H', 'I', 'G', 'F']


In [30]:
# Practice: iterative traverse a tree: in-order

def iter_inorder(root):
    # 和post-order traverse类似
    # 在post-order中搜索tree和访问tree是分离的
    # 需要分别执行搜索和访问
    track = []
    s = list()
    s.append(root)
    visited = set([None])
    
    while len(s) > 0:
        root = s[-1]
        
        if root == None:
            return []
        
        # 如果左边访问了，那就访问自己
        if root.left in visited:
            s.pop()
            track.append(root.val)
            visited.add(root)
            # 只有在访问了自己之后才有必要探索右边
            if root.right != None:
                s.append(root.right)
                
        # 如果左边没有访问，那就把左边加到stack上面准备访问
        else:
            s.append(root.left)
    
    return track

In [32]:
# Test iterative travers a tree post-order
root = de_preorder(pre_order)
track = iter_inorder(root)
print track

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']


In [64]:
# Lowest common ancestor

def lowestCommonAncestor(root, p, q):
    """
    :type root: TreeNode
    :type p: value
    :type q: value
    :rtype: TreeNode
    """
    
    def recur_LCA(root, p, q):
        
        # If the root is none
        if root == None:
            return None, False
        
        
        # Get result from left and right, even if left and right child is None, this is None-safe
        left_res, left_isLCA = recur_LCA(root.left, p, q)
        right_res, right_isLCA = recur_LCA(root.right, p, q)
        
        # If left or right finds the LCA
        if left_isLCA: return left_res, True
        if right_isLCA: return right_res, True
        
        
        # Now we know we haven't found any LCA yet
        
        # If both children are nodes, then root is LCA
        if isinstance(left_res, Node) and isinstance(right_res, Node):
            return root, True
        
        # If only one of the child is node, 
        elif isinstance(left_res, Node) or isinstance(right_res, Node):
            # if root itself is a target, then root is LCA
            if root.val == p or root.val == q:
                return root, True
            # else return the node that one of child returns
            else:
                if isinstance(left_res, Node):
                    return left_res, False
                else:
                    return right_res, False
        
        # None of the child finds the target
        else:
            if root.val == p or root.val == q:
                return root, False
            else:
                return None, False
    
    node, is_LCA = recur_LCA(root, p, q)
    if is_LCA:
        return node
    else:
        return None

In [65]:
# Test lowest commoan ancestor

pre_order = ["F","B","A","#","#","D","C","#","#","E","#","#","G","#","I","H","#","#","#"]
root = de_preorder(pre_order)

print lowestCommonAncestor(root, "B", "G").val
print lowestCommonAncestor(root, "B", "F").val
print lowestCommonAncestor(root, "C", "E").val
print lowestCommonAncestor(root, "A", "B").val
print lowestCommonAncestor(root, "A", "E").val
print lowestCommonAncestor(root, "A", "H").val
print lowestCommonAncestor(root, "X", "Y")
print lowestCommonAncestor(root, "A", "Z")

F
F
D
B
B
F
None
None


In [ ]:
# Lowest Common Ancestor II: using path and find the path
# Very slow becuase on the returning, each list is concatenated  

def find_path(self, root, p):
        if root == p:
            return [root]
        
        if root.left == None and root.right == None:
            return []
        
        if root.left != None:
            res = self.find_path(root.left, p)
            if res:
                return [root] + res
        
        if root.right != None:
            res = self.find_path(root.right, p)
            if res:
                return [root] + res
        
        return []
    
    def lowestCommonAncestor(self, root, p, q):
        """
        :type root: TreeNode
        :type p: TreeNode
        :type q: TreeNode
        :rtype: TreeNode
        """
        
        path_p = self.find_path(root, p)
        
        path_q = self.find_path(root, q)
        
        if len(path_p) == 0 or len(path_q) == 0:
            return None
        
        for i in range(len(path_p)-1, -1, -1):
            if path_p[i] in path_q:
                return path_p[i]
        
        return None

In [ ]:
# Lowest Common Ancestor III
# Use only one path all the time. Keep using the back tracking

def find_path(self, root, p):
        
        def _recur(root, p):
            path.append(root)
            
            if path[-1] == p:
                return True
            
            if root.left == None and root.right == None:
                path.pop()
                return False
            
            if root.left != None:
                res = _recur(root.left, p)
                if res == True:
                    return True
            
            if root.right != None:
                res = _recur(root.right, p)
                if res == True:
                    return True
            
            # Not found, need to remove added root
            path.pop()
            return False
        
        if root == None:
            print "root is None! Empty path returned!"
            return []
            
        path = []
        res = _recur(root, p)
        if res == True:
            print "Find path from {} to {}".format(root.val, p.val)
        else:
            print "Can't find path from {} to {}".format(root.val, p.val)
        return path
    
    def lowestCommonAncestor(self, root, p, q):
        """
        :type root: TreeNode
        :type p: TreeNode
        :type q: TreeNode
        :rtype: TreeNode
        """
        
        path_p = self.find_path(root, p)
        
        path_q = self.find_path(root, q)
        
        for i in range(len(path_p)-1, -1, -1):
            if path_p[i] in path_q:
                return path_p[i]
        
        return None

In [5]:
# Morris In-order Traversal (O(1) space)
# Build thread to help go to next node in-order
# Destroy the thread after use

"""
1. Initialize current as root 
2. While current is not NULL
   If current does not have left child
      a) Print current’s data
      b) Go to the right, i.e., current = current->right
   Else
      a) Make current as right child of the rightmost node in current's left subtree
      b) Go to this left child, i.e., current = current->left
"""

def se_morris_inorder(root):
    lst = []
    cur = root        
    
    while cur != None:
        
        # If current doesn't have left, time to serialize current, then move to right tree
        # This is the same logic in iterative traversal using stack as well
        if cur.left == None:
            lst.append(cur.val)
            cur = cur.right # this step including using thread to go to next node
        
        # If current has left tree, before traverse it, need to ensure after dealing with left,
        # the traversal can go back to current so it can go to right. 
        # To do that, need to build a thread from rightmost in the left tree to current node  
        else:
            pre = cur.left
            # Use while loop to go to rightmost node in the left tree
            while pre.right != None and pre.right != cur:
                pre = pre.right
            
            # Now 2 situations: pre doesn't have a thread yet or pre already has thread
            
            # Situation 1 - doesn't have a thread yet: add a thread and start to deal with left tree
            if pre.right == None:
                pre.right = cur
                cur = cur.left # Then the new outmost while loop will start traverse 
            
            # Situation 2 - does have a thread: 
            #     It's 2nd visiting this node, that means we are 2nd time at the current node
            #     which in turn means we have used the thread, which means we have done visiting the previous node 
            #     So we can visit the current node,  move to next node, and destroy the thread before leaving
            else:
                lst.append(cur.val) # change this line to if, in-order becomes pre-order traversal!!!
                cur = cur.right
                pre.right = None
    
    return lst

In [7]:
# Test Morris In-order Traversal
root = de_preorder(pre_order)
print(se_morris_inorder(root))

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
